### Parameters to update
- **FabricDWWorkspaceName**: The name of the workspace that the warehouse exists.
- **FabricDWName**: The name of the warehouse.
- **ConcurrnecyNum**: The number of queries that will be executings in parallel.
- **CapacityMetricsWorkspace**: The name of the workspace that the capacity metrics semantic model exists.
- **CapacityMetricsDataset**: The name of the capacity metrics app semenatic model.
- **StoreQueryResults**: Flag to set if the results of the queries will be stored in the query results table.
- **QueryRepeatCount**: Number of times a query will run (should be between 1 and 4) eg. QueryRepeatCount = 4 and queryList = [query1, query2] will become [query1, query1, query1, query1, query2, query2, query2]
- **RunName**: The name of the run. If not specified, one will be generated with the following format. '*Run_{yyyyMMdd}_{hhmmss}*'

In [ ]:
FabricDWWorkspaceName = ''
FabricDWName = 'WH_SampleData'
ConcurrencyNum = 1 # This should be equal or less than the length of the list with the queries defined below
CapacityMetricsWorkspace = 'Microsoft Fabric Capacity Metrics'
CapacityMetricsDataset = 'Fabric Capacity Metrics'
StoreQueryResults = False
QueryRepeatCount = 4 # Number of times a query will run eg. queryRepeatCount = 4, queryList = [query1, query2] will become [query1, query1, query1, query1, query2, query2, query2]
RunName = '' # The name of the run. If not specified, one will be generated with the following format. '*Run_{yyyyMMdd}_{hhmmss}*'

In [ ]:
import datetime
RunName = RunName if RunName else f"Run_{datetime.datetime.now(datetime.timezone.utc).strftime('%Y%m%d_%H%M%S')}"
print(f"{FabricDWWorkspaceName=}")
print(f"{FabricDWName=}")
print(f"{ConcurrencyNum=}")
print(f"{CapacityMetricsWorkspace=}")
print(f"{CapacityMetricsDataset=}")
print(f"{StoreQueryResults=}")
print(f"{RunName=}")

In [ ]:
queryList = [
    # Transaction
    'SELECT COUNT(*) FROM FactTransaction'
    ,'''SELECT	COUNT(*)
FROM	FactTransaction AS ft
JOIN	DimDate AS d
ON		d.Date = ft.DateKey
JOIN	DimPaymentMethod AS pm
ON		pm.PaymentMethodKey= ft.PaymentMethodKey
JOIN	DimTransactionType AS tt
ON		tt.TransactionTypeKey = ft.TransactionTypeKey
JOIN	DimSupplier AS s
ON		s.SupplierKey = ft.SupplierKey
JOIN	DimCustomer AS cu
ON		cu.CustomerKey = ft.CustomerKey
JOIN	DimCustomer AS cuBill
ON		cuBill.CustomerKey = ft.BillToCustomerKey'''
    # Order
    ,'SELECT COUNT(*) cnt_FactOrder FROM FactOrder'
    ,'''SELECT	COUNT(*)
FROM	FactOrder AS fo
JOIN	DimDate AS dOrder
ON		dOrder.Date = fo.OrderDateKey
LEFT JOIN	DimDate AS dPicked
ON		dPicked.Date = fo.PickedDateKey
JOIN	DimStockItem AS si
ON		si.StockItemKey = fo.StockItemKey
JOIN	DimCity AS c
ON		c.CityKey = fo.CityKey
JOIN	DimEmployee AS e
ON		e.EmployeeKey = fo.SalespersonKey
JOIN	DimEmployee AS ePicker
ON		ePicker.EmployeeKey = fo.PickerKey
JOIN	DimCustomer AS cu
ON		cu.CustomerKey = fo.CustomerKey'''
    # FactMovement
    ,'SELECT COUNT(*) cnt_FactMovement FROM FactMovement'
    ,'''SELECT	COUNT(*)
FROM	FactMovement AS fm
JOIN	DimDate AS d
ON		d.Date = fm.DateKey
JOIN	DimStockItem AS si
ON		si.StockItemKey = fm.StockItemKey
JOIN	DimTransactionType AS tt
ON		tt.TransactionTypeKey = fm.TransactionTypeKey
JOIN	DimSupplier AS s
ON		s.SupplierKey = fm.SupplierKey
JOIN	DimCustomer AS c
ON		c.CustomerKey = fm.CustomerKey'''
    # FactPurchase
    ,'SELECT COUNT(*) cnt_FactPurchase FROM FactPurchase'
    ,'''SELECT	COUNT(*)
FROM	FactPurchase AS fp
JOIN	DimDate AS d
ON		d.Date = fp.DateKey
JOIN	DimStockItem AS si
ON		si.StockItemKey = fp.StockItemKey
JOIN	DimSupplier AS s
ON		s.SupplierKey = fp.SupplierKey'''
    # StockHolding
    ,'SELECT COUNT(*) AS cnt_FactStockHolding FROM FactStockHolding'
    ,'''SELECT	COUNT(*)
FROM	FactStockHolding AS fsh
JOIN	DimStockItem AS si
ON		si.StockItemKey = fsh.StockItemKey'''
    ,'''SELECT	TOP 100 fsh.*
FROM	FactStockHolding AS fsh
JOIN	DimStockItem AS si
ON		si.StockItemKey = fsh.StockItemKey'''
    # Stored Procedures
    ,'''EXEC sp_Ingest'''
    ,'''EXEC sp_Query'''
    # Query with multiple statements
    ,'''IF OBJECT_ID('dbo.DimDate', 'U') IS NOT NULL DROP TABLE dbo.DimDate; CREATE TABLE dbo.DimDate AS SELECT * FROM LH_SampleData.dbo.DimDate'''
]

In [ ]:
executorCoreCnt = int(spark.conf.get('spark.executor.cores', '0'))
executorInstances = len(spark._jsc.sc().statusTracker().getExecutorInfos()) - 1
maxConcurrency = ConcurrencyNum if ConcurrencyNum < (executorCoreCnt * executorInstances) else (executorCoreCnt * executorInstances)

# Adding queries to the queryList so that each queries executes 4 times sequentially.
# Example, original queryList = [query_1, query_2] becomes [query_1, query_1, query_1, query_1, query_2, query_2, query_2, query_2]
queryListWithRepeat = []
for query in queryList:
    queryListWithRepeat.extend([query] * QueryRepeatCount)

# Multipling the number of queries by the maxConcurrency to create a pool of queries to be executed.
# This will make it so that each number of concurrency will execute every query in the queryList.
# Eg. If the queryList has 10 queries and the maxConcurrency is 5, then the queryPool will have 50 queries.
queryPool = queryListWithRepeat * maxConcurrency

rddQueries = sc.parallelize(queryPool, maxConcurrency)
rddQueriesWithId = rddQueries.zipWithUniqueId().map(lambda x: [x[1], (x[1], x[0])] )
rddQueriesWithId = rddQueriesWithId.partitionBy(maxConcurrency, lambda k: k ) 
print(rddQueriesWithId.glom().map(len).collect())  # Check the length of each partition to check for even distribution of rows in the partitions. This will tell us if the number of queries are evenly distributed

displayHTML(f"""
<p><span style="font-size:20px;"><strong>Max concurrency of spark session is </strong><i><strong>{executorCoreCnt * executorInstances}</strong></i></span></p>
<p><span style="font-size:20px;"><strong>Defined concurrency is </strong><i><strong>{ConcurrencyNum}</strong></i></span></p>
<p><span style="font-size:20px;"><strong>Will run </strong><i><strong>{maxConcurrency}</strong></i><strong> queries concurrently for this spark session</strong></span></p>
<p><span style="font-size:20px;"><strong>A total of <i>{len(queryPool)}</i> queries will be executed (queryList size <i>{len(queryList)}</i> * max concurrency <i>{maxConcurrency}</i> * query repeat count <i>{QueryRepeatCount}</i>)</strong></span></p>
""")

In [ ]:
import requests

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces', headers=header)

while True:
    workspaceFound = False
    for workspace in response.json().get('value'):
        if workspace.get('displayName') == FabricDWWorkspaceName:
            fabricDWWorkspaceId = workspace.get('id')
            workspaceFound = True
            break
    
    if workspaceFound:
        break
    elif workspaceFound == False and response.json().get('continuationToken'):
        responseStatus = requests.request(method='get', url=response.json().get('continuationUri'), headers=header)
    else:
        print(f"Workspace was not found and no contination token found - {response.json()}")
        break

print(f'{fabricDWWorkspaceId = }\n{FabricDWWorkspaceName = }')

In [ ]:
import requests, json

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{fabricDWWorkspaceId}/items', headers=header)

for item in response.json().get('value'):
    if item.get('displayName') == FabricDWName:
        itemType = item.get('type')
        
print(f'{itemType = }')

In [ ]:
import requests

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{fabricDWWorkspaceId}', headers=header)
workspaceName = response.json().get('displayName')

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{fabricDWWorkspaceId}', headers=header)
capacityId = response.json().get('capacityId')
capacityRegion = response.json().get('capacityRegion')
capacityName = response.json().get('displayName')
capacitySku = 'F0' #Default value of F0

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/capacities', headers=header)
for capacity in response.json().get('value'):
    if capacity.get('id') == capacityId:
        capacitySku = capacity.get('sku')

if itemType == 'Lakehouse':
    response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{fabricDWWorkspaceId}/lakehouses', headers=header)
    warehouse = [warehouse for warehouse in response.json().get('value') if warehouse.get('displayName') == FabricDWName][0]
    fabricDWServer = warehouse.get('properties').get('sqlEndpointProperties').get('connectionString')
else:
    response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{fabricDWWorkspaceId}/warehouses', headers=header)
    warehouse = [warehouse for warehouse in response.json().get('value') if warehouse.get('displayName') == FabricDWName][0]
    fabricDWServer = warehouse.get('properties').get('connectionString')
    
warehouseId = warehouse.get('id')

print(f'{warehouseId = }\n{fabricDWServer = }\n{workspaceName = }\n{capacityId = }\n{capacityRegion = }\n{capacityName = }\n{capacitySku = }')

In [ ]:
response = requests.request(method='get', url="https://prices.azure.com/api/retail/prices?$filter=skuName eq 'Fabric Capacity'", headers=header)
for capacity in response.json().get('Items'):
    if capacity.get('armRegionName') == capacityRegion.replace(' ', '').lower():
        costReserved = capacity.get('retailPrice') / 12 / 730 / 60 / 60 # get the amount per CU second
        costPayGo = costReserved / (156.334/262.80) # constant saving of ~41%. 156.334 is the resevered price of a region. 262.80 is the paygo price of a region
print(f'{costReserved = :.10f}\n{costPayGo = :.10f}') # per CU

##### Define the queries to be executed. These are single line queries so use /* */ for commenting out code vs --

In [ ]:
from notebookutils import mssparkutils  
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, TimestampType, DoubleType
import pyodbc, struct, itertools, time, datetime, re, uuid, json

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={fabricDWServer};Database={FabricDWName};APP=QueryCostAnalyzer'

# Use the credentials of the user executing the notebook
token = bytes(mssparkutils.credentials.getToken('pbi'), "UTF-8")
encoded_bytes = bytes(itertools.chain.from_iterable(zip(token, itertools.repeat(0))))
tokenstruct = struct.pack("<i", len(encoded_bytes)) + encoded_bytes

runId = str(uuid.uuid4()).upper()

print(f'{runId = }')

In [ ]:
from delta.tables import *

with pyodbc.connect(connectionString, attrs_before = { 1256:tokenstruct }) as conn:
    with conn.cursor() as cursor:
        cursor.execute('''SELECT @@VERSION AS DWVersion
                            ,@@SERVERNAME AS ServerGuid
                            ,DB_NAME() AS DWName
        ''')
        resultList = cursor.fetchall()
        resultColumns = [column[0] for column in cursor.description]
        cursor.commit()
        resultSet = [dict(zip(resultColumns, [str(col) for col in row])) for row in resultList]

        cursor.execute(f'''SELECT [is_vorder_enabled] AS IsVOrderEnabled, [data_lake_log_publishing_desc] AS DataLakeLogPublishingDesc
                            ,[data_lake_log_publishing] AS DataLakeLogPublishing, [create_date] AS DWCreateDate, [compatibility_level] AS CompatibilityLevel
                            FROM sys.databases 
                            WHERE [name] = '{FabricDWName}'
            ''')

        resultList = cursor.fetchall()
        resultColumns = [column[0] for column in cursor.description]
        cursor.commit()
        resultSet = resultSet[0] | [dict(zip(resultColumns, [str(col) for col in row])) for row in resultList][0]
        df = spark.createDataFrame([resultSet])

dfRun = (df.withColumn('RunStartDateTimeUTC', F.lit(None).cast(TimestampType()))
            .withColumn('RunStartTimeEpochMS', F.lit(None).cast(LongType()))
            .withColumn('RunName', F.lit(RunName).cast(StringType()))
            .withColumn('RunId', F.lit(runId).cast(StringType()))
            .withColumn('DWConnectionString', F.lit(fabricDWServer).cast(StringType()))
            .withColumn('QueriesExecutedCnt', F.lit(len(queryPool)).cast(IntegerType()))
            .withColumn('RunConcurrency', F.lit(maxConcurrency).cast(IntegerType()))
            .withColumn('QueryRepeatCount', F.lit(QueryRepeatCount).cast(IntegerType()))
            .withColumn('StoreQueryResults', F.lit(StoreQueryResults).cast(StringType()))

            .withColumn('DWGuid', F.lit(warehouseId).cast(StringType())) 
            .withColumn('WorkspaceName', F.lit(workspaceName).cast(StringType())) 
            .withColumn('WorkspaceGuid', F.lit(fabricDWWorkspaceId).cast(StringType())) 
            .withColumn('CapacityName', F.lit(capacityName).cast(StringType())) 
            .withColumn('CapacityGuid', F.lit(capacityId).cast(StringType())) 
            .withColumn('CapacitySKU', F.lit(capacitySku).cast(StringType())) 
            .withColumn('CapacityRegion', F.lit(capacityRegion).cast(StringType())) 

            .withColumn('RunEndDatetimeUTC', F.lit(None).cast(TimestampType()))
            .withColumn('RunEndTimeEpochMS', F.lit(None).cast(LongType()))
            .withColumn('RunDurationMS', F.lit(None).cast(LongType()))
            .withColumn('RunCUSeconds', F.lit(None).cast(DoubleType()))
            .withColumn('RunCostPayGo', F.lit(None).cast(DoubleType()))
            .withColumn('RunCostReserved', F.lit(None).cast(DoubleType()))
            
            .withColumn('CapacityDailyCUSeconds', F.lit(60*60*24*int(64 if ''.join([str(i) for i in capacitySku if i.isdigit()]) == 1 else ''.join([str(i) for i in capacitySku if i.isdigit()])) ))
            .withColumn('CapacityDailyCostPayGo', F.lit(costPayGo * 60*60*24*int(64 if ''.join([str(i) for i in capacitySku if i.isdigit()]) == 1 else ''.join([str(i) for i in capacitySku if i.isdigit()])) ))
            .withColumn('CapacityDailyCostReserved', F.lit(costReserved * 60*60*24*int(64 if ''.join([str(i) for i in capacitySku if i.isdigit()]) == 1 else ''.join([str(i) for i in capacitySku if i.isdigit()])) ))
        )

dfRunOrdered = dfRun.select("RunName", "RunId", "DWConnectionString", "QueriesExecutedCnt", "RunConcurrency", "QueryRepeatCount", "StoreQueryResults", "ServerGuid"
        , "DWGuid", "DWName", "DWVersion", "WorkspaceName", "WorkspaceGuid", "CapacityName", "CapacityGuid", "CapacitySKU", "CapacityRegion", "CompatibilityLevel"
        , "DWCreateDate", "DataLakeLogPublishing", "DataLakeLogPublishingDesc", "IsVOrderEnabled"
        , "RunStartDateTimeUTC", "RunStartTimeEpochMS", "RunEndDatetimeUTC", "RunEndTimeEpochMS", "RunDurationMS", "RunCUSeconds"
        , "RunCostPayGo", "RunCostReserved", "CapacityDailyCUSeconds", "CapacityDailyCostPayGo", "CapacityDailyCostReserved"
    )

if spark.catalog.tableExists("RunResults"):

    dtRunResults = DeltaTable.forName(spark, "RunResults")
    
    (dtRunResults.alias('t')
        .merge(dfRunOrdered.alias('s')
            ,f't.runId = s.RunId'
            )
        .whenNotMatchedInsertAll()
    ).execute() 
else:
    dfRunOrdered.write.format('delta').mode('append').saveAsTable('RunResults')

In [ ]:
from pyspark import SparkContext, SparkConf
import pyodbc 

def get_result_set(cursor):
    if cursor.description:
        resultList = cursor.fetchall()
        resultRowCnt = len(resultList)
        resultColumns = []
        if StoreQueryResults:
            resultColumns = [column[0] for column in cursor.description]
    else:
        resultList = []
        resultColumns = []
        resultRowCnt = 0
    return [dict(zip(resultColumns, [str(col) for col in row])) for row in resultList], resultRowCnt

def execute_query(iterator):
    queryMetrics = []
    for queryInfo in iterator:
        queryIndex = queryInfo[1][0]
        queryStatement = queryInfo[1][1]
        with pyodbc.connect(connectionString, attrs_before = { 1256:tokenstruct }) as conn:
            with conn.cursor() as cursor:
                queryStartDateTimeUTC = datetime.datetime.now(datetime.timezone.utc)
                startTime = int(round(time.time() * 1000))

                cursor.execute(queryStatement)
                
                queryMessage = str(cursor.messages) if cursor.messages else ""
                resultSetList = list()
                resultRowCntList = list()
                resultSet, resultRowCnt = get_result_set(cursor)
                resultSetList.append(resultSet)
                resultRowCntList.append(resultRowCnt)

                while cursor.nextset():
                    queryMessage += ",".join([str(cursor.messages) if cursor.messages else ""])
                    resultSet, resultRowCnt = get_result_set(cursor)
                    resultSetList.append(resultSet)
                    resultRowCntList.append(resultRowCnt)
                    
                endTime = int(round(time.time() * 1000))
                queryEndDateTimeUTC = datetime.datetime.now(datetime.timezone.utc)
                
                statementId = ','.join(re.findall(r"Statement ID: \{([A-F0-9\-]+)\}", queryMessage)) if re.findall(r"Statement ID: \{([A-F0-9\-]+)\}", queryMessage) else ""
                queryHash = ','.join(re.findall(r"Query hash: (0x[A-F0-9]+)", queryMessage)) if re.findall(r"Query hash: (0x[A-F0-9]+)", queryMessage) else ""
                distributionRequestId = ','.join(re.findall(r"Distributed request ID: \{([A-F0-9\-]+)\}", queryMessage)) if re.findall(r"Distributed request ID: \{([A-F0-9\-]+)\}", queryMessage) else ""
                resultSetJsonString = json.dumps(resultSetList)

                cursor.commit()

                queryId = str(uuid.uuid4()).upper()
                queryMetrics.append([runId, queryId, queryStatement, queryStartDateTimeUTC, queryEndDateTimeUTC
                        ,queryMessage, startTime, endTime, endTime - startTime
                        ,statementId, queryHash, distributionRequestId, resultSetJsonString, resultRowCntList
                        ])
    return queryMetrics

queriesExecuted = rddQueriesWithId.mapPartitions(execute_query)

runStartDateTimeUTC = datetime.datetime.now(datetime.timezone.utc)
runStartTimeEpoch = int(runStartDateTimeUTC.timestamp()*1000)

queriesExecuted.cache().count()

runEndDateTimeUTC = datetime.datetime.now(datetime.timezone.utc)
runEndTimeEpoch = int(runEndDateTimeUTC.timestamp()*1000)

dfQueriesExecuted = queriesExecuted.toDF(schema=StructType([
    StructField("RunId", StringType(), False),
    StructField("QueryId", StringType(), False),
    StructField("QueryStatement", StringType(), False),
    StructField("QueryStartDateTimeUTC", TimestampType(), False),
    StructField("QueryEndDateTimeUTC", TimestampType(), False),
    StructField("ReturnMessage", StringType(), False),
    StructField("QueryStartTimeEpochMS", LongType(), False),
    StructField("QueryEndTimeEpochMS", LongType(), False),
    StructField("QueryDurationMS", LongType(), False),
    StructField("StatementId", StringType(), False),
    StructField("QueryHash", StringType(), False),
    StructField("DistributionRequestId", StringType(), False),
    StructField("ResultSet", StringType(), False),
    StructField("ResultRowCnt", StringType(), False)
    ]))

dfFinal = dfQueriesExecuted.withColumn('QueryCUSeconds', F.lit(None).cast(DoubleType())).withColumn('QueryCostPayGo', F.lit(None).cast(DoubleType())).withColumn('QueryCostReserved', F.lit(None).cast(DoubleType()))
dfFinal.write.format('delta').mode('append').saveAsTable('QueryResults')

In [ ]:
statementList = spark.sql(f'SELECT ARRAY_JOIN(COLLECT_SET(CONCAT("\\"", StatementId, "\\"")), ", ") AS Statements FROM (SELECT EXPLODE(SPLIT(StatementId, ",")) AS StatementId FROM QueryResults WHERE runId = "{runId}") AS a ').collect()[0].asDict().get('Statements')
# We have to explode by statement ids since a sql query may have multiple queries within it
queriesExecutedCnt = spark.sql(f'SELECT COUNT(StatementId) AS QueryCnt FROM (SELECT EXPLODE(SPLIT(StatementId, ",")) AS StatementId FROM QueryResults WHERE runId = "{runId}") AS a ').collect()[0].asDict().get('QueryCnt') 
print(f'{runId = }\n{statementList = }\n{queriesExecutedCnt = }')

In [ ]:
from delta.tables import *

dtRunResults = DeltaTable.forName(spark, "RunResults")

df_final = spark.createDataFrame(data=[(runStartDateTimeUTC, runStartTimeEpoch, runEndDateTimeUTC, runEndTimeEpoch, )], schema=['runStartDateTimeUTC', 'runStartTimeEpochMS', 'RunEndDateTimeUTC', 'RunEndTimeEpochMS'])

(dtRunResults.alias('t')
    .merge(df_final.alias('s')
        ,f't.RunId = "{runId}"'
        )
    .whenMatchedUpdate(set=
        {'RunStartDateTimeUTC': 's.RunStartDateTimeUTC'
        ,'RunStartTimeEpochMS': 's.RunStartTimeEpochMS'
        ,'RunEndDatetimeUTC': 's.RunEndDateTimeUTC'
        ,'RunEndTimeEpochMS': 's.RunEndTimeEpochMS'
        ,'RunDurationMS': 's.RunEndTimeEpochMS - s.RunStartTimeEpochMS'
        }
        )
).execute() 

In [ ]:
import requests, datetime
from pyspark.sql.functions import to_timestamp, lit, col

def get_capacity_metrics_usage(time_point:datetime, operation_id_list:str):

    schema = StructType([
        StructField("Items[ItemId]", 	StringType(), 		True),
        StructField("Items[ItemKind]", 	StringType(), 		True),
        StructField("Items[ItemName]", 	StringType(), 		True),
        StructField("TimePointBackgroundDetail[OperationStartTime]", 	StringType(), 	True),
        StructField("TimePointBackgroundDetail[OperationEndTime]", 		StringType(), 	True),
        StructField("TimePointBackgroundDetail[OperationId]", 			StringType(), 		True),
        StructField("[Sum_CUs]", 		DoubleType(), 		True),
        StructField("[Sum_Duration]", IntegerType(), 		True)
    ])

    header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
                ,"Content-Type": "application/json"
                }

    response = requests.get('https://api.fabric.microsoft.com/v1/workspaces', headers=header)

    capacityWorkspaceId = [workspace.get('id') for workspace in response.json().get('value') if workspace.get('displayName') == CapacityMetricsWorkspace][0]

    response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets", headers=header)

    datasetId = [dataset.get('id') for dataset in response.json().get('value') if dataset.get('name') == CapacityMetricsDataset][0]
    
    body = {
        "queries": [
        {
            "query": f"""
                DEFINE
                    MPARAMETER 'CapacityID' 	= "{capacityId}"
                    MPARAMETER 'TimePoint' 		= (DATE({time_point.year}, {time_point.month}, {time_point.day}) + TIME({time_point.hour}, {time_point.minute}, {time_point.second}))

                    VAR __Var_CapacityId	= {{"{capacityId}"}}
                    VAR __Var_OperationId	= {{{statementList}}}

                    VAR __Filter_OperationId 	= TREATAS(__Var_OperationId, 'TimePointBackgroundDetail'[OperationId])
                    VAR __Filter_CapacityId 	= TREATAS(__Var_CapacityId, 'Capacities'[capacityId])

                    VAR OperationCUs = 
                        SUMMARIZECOLUMNS(
                            'Items'[ItemId],
                            'Items'[ItemKind],
                            'Items'[ItemName],
                            'TimePointBackgroundDetail'[OperationStartTime],
                            'TimePointBackgroundDetail'[OperationEndTime],
                            'TimePointBackgroundDetail'[OperationId],
                            __Filter_OperationId,
                            __Filter_CapacityId,
                            "Sum_CUs", CALCULATE(SUM('TimePointBackgroundDetail'[Total CU (s)])),
                            "Sum_Duration", CALCULATE(SUM('TimePointBackgroundDetail'[Duration (s)]))
                        )

                EVALUATE
                    OperationCUs
                """
        }
        ]
    }

    response = requests.post(f'https://api.powerbi.com/v1.0/myorg/datasets/{datasetId}/executeQueries', headers=header, json=body )

    if response.status_code == 200:
        rowsList = response.json()["results"][0]["tables"][0]["rows"]
        
        df_dax = spark.createDataFrame(data=rowsList, schema=schema)
        df_dax = df_dax.select('*', to_timestamp('TimePointBackgroundDetail[OperationStartTime]'), to_timestamp('TimePointBackgroundDetail[OperationEndTime]'))
        
        df = (df_dax.withColumn("TimePoint", lit(time_point)).select(
            col("TimePoint")
            ,col("Items[ItemId]").alias("ItemId")
            ,col("Items[ItemKind]").alias("ItemKind")
            ,col("Items[ItemName]").alias("ItemName")
            ,col("TimePointBackgroundDetail[OperationStartTime]").alias("StartTime")
            ,col("TimePointBackgroundDetail[OperationEndTime]").alias("EndTime")
            ,col("TimePointBackgroundDetail[OperationId]").alias("OperationId")
            ,col("[Sum_CUs]").cast(DoubleType()).alias("Sum_CUs")
            ,col("[Sum_Duration]").cast(IntegerType()).alias("Sum_Duration"))
        )

        return df
    else:
        raise Exception(f'{response.json()}\nCheck that user has as least contributor access to the workspace')

In [ ]:
import requests, time

def model_refresh():
    header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
            ,"Content-Type": "application/json"
            }

    response = requests.get('https://api.fabric.microsoft.com/v1/workspaces', headers=header)

    capacityWorkspaceId = [workspace.get('id') for workspace in response.json().get('value') if workspace.get('displayName') == CapacityMetricsWorkspace][0]

    response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets", headers=header)

    datasetId = [dataset.get('id') for dataset in response.json().get('value') if dataset.get('name') == CapacityMetricsDataset][0]

    response = requests.post(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes", headers=header)

    refreshId = response.headers.get('RequestId')
    print(f'{refreshId = } | {response.status_code = }')

    if response.status_code == 202:
        for attempt in range(12): 
            # https://learn.microsoft.com/en-us/power-bi/connect-data/asynchronous-refresh#get-refreshes
            response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes?$top=1", headers=header)
            if response.status_code == 200:
                if response.json().get('value')[0].get('status') != 'Unknown':
                    print(f'Refresh Complete')
                    break
                else:
                    print(f'Refreshing tables ...')
                    time.sleep(20)
            else:
                time.sleep(10)
    else:
        print(f'Refreshed failed - {response.text}')

In [ ]:
'''
Get the capacity usage for the queries that were executed.
This ensures that the background operations are captured no matter what time they are run as they are smoothed over a 24 hour time period.
Next, filter that down to the distinct records. This is necessary because a record may show up in the today and tomorrow datasets depending on the time it was run.
Nest, aggregate the records into a single record. This is necessary becuase some operations will have two entries, one under the executing user and one under the user "System".
'''
from pyspark.sql.functions import min, max, sum

# # Continues to query the metrica app to get the data. Data can delayed by a few minutes.
# # We retry every minute until 15 minutes has passed.
for retryCnt in range(15):
    df_today = get_capacity_metrics_usage(runStartDateTimeUTC, statementList)
    df_tomorrow = get_capacity_metrics_usage(runStartDateTimeUTC + datetime.timedelta(hours = 23), statementList)
    df_all_days = df_today.unionAll(df_tomorrow)
    df_count = df_all_days.select('OperationId').distinct()
    
    print(f'{df_count.count()} statements of the {queriesExecutedCnt} that have been found in the capacity metrics model. ', end='')
    if df_count.count() == queriesExecutedCnt:
        df_final = df_all_days.groupBy('ItemId', 'ItemKind', 'ItemName', 'OperationId').agg(min("StartTime").alias("StartTime"), max("EndTime").alias("EndTime"), sum("Sum_CUs").alias("QueryCUSeconds"), sum("Sum_Duration").alias("SumDuration"))
        break
    else:
        if retryCnt%5 == (5-1):
            print('Refreshing the capacity metircs app semantic model...')
            model_refresh()
            continue
        print('Sleeping for a minute...')
        time.sleep(60)

In [ ]:
from pyspark.sql.functions import sum

dfQueryResults = spark.table("QueryResults")

dfQueryResultsCleansed = (dfQueryResults.join(df_final, dfQueryResults.StatementId.contains(df_final.OperationId)) 
    .filter(dfQueryResults.RunId == runId) 
    .groupBy(dfQueryResults.RunId, dfQueryResults.StatementId) 
    .agg(sum(df_final.QueryCUSeconds).alias("QueryCUSeconds"), sum(df_final.SumDuration).alias("SumDuration")
    ))

##### Update the QueryResults table with the CUSeconds and QueryCost derived from the Capacity Metrics App

In [ ]:
from delta.tables import *

dtQueryResults = DeltaTable.forName(spark, "QueryResults")

(dtQueryResults.alias('t')
    .merge(dfQueryResultsCleansed.alias('s')
        ,f't.runId = s.RunId AND t.StatementId = s.StatementId'
        )
    .whenMatchedUpdate(set=
        {'QueryCUSeconds': 's.QueryCUSeconds'
        ,'QueryCostPayGo': f's.QueryCUSeconds * {costPayGo}'
        ,'QueryCostReserved': f's.QueryCUSeconds * {costReserved}'
        }
        )
).execute() 

##### Update the RunResults table with cost of run

In [ ]:
from delta.tables import *

dtRunResults = DeltaTable.forName(spark, "RunResults")
dtRunResultsCleansed = spark.sql(f'SELECT SUM(COALESCE(QueryCUSeconds, 0)) AS RunCUSeconds, SUM(COALESCE(QueryCostPayGo, 0)) AS RunCostPayGo, SUM(COALESCE(QueryCostReserved, 0)) AS RunCostReserved FROM QueryResults WHERE RunId = "{runId}"')

(dtRunResults.alias('t')
    .merge(dtRunResultsCleansed.alias('s')
        ,f't.RunId = "{runId}"'
        )
    .whenMatchedUpdate(set=
        {'RunCUSeconds': 's.RunCUSeconds'
        ,'RunCostPayGo': f's.RunCUSeconds * {costPayGo}'  # This could be different looking at the tables separately due to rounding
        ,'RunCostReserved': f's.RunCUSeconds * {costReserved}'  # This could be different looking at the tables separately due to rounding
        }
        )
).execute()

In [ ]:
display(spark.sql(f"SELECT * FROM RunResults WHERE RunId = '{runId}'"))

In [ ]:
display(spark.sql(f"SELECT * FROM QueryResults WHERE RunId = '{runId}'"))

In [ ]:
from delta.tables import *

spark.conf.set('spark.databricks.delta.retentionDurationCheck.enabled', False)

for tableName in ['QueryResults', 'RunResults']:
    dt = DeltaTable.forName(spark, tableName)
    dt.optimize().executeCompaction()
    dt.vacuum(1)

In [1]:
displayHTML(f"""<script src="https://cdn.rawgit.com/google/code-prettify/master/loader/run_prettify.js"></script>
<p style="margin-bottom:0"><span style="font-size:20px;"><strong>/*<br>Reference T-SQL - </strong></span><span style="font-size:20px;"><strong>See running sql statements on the DW. Used to verify query(s) are executing and that the concurrency is working correctly.<br>*/</strong></span></p>
<pre class="prettyprint"><p style="margin-top:0;">SELECT	d.name AS 'database_name'
	,s.login_name
	,r.[session_id]
	,r.start_time
	,r.STATUS
	,r.total_elapsed_time
	,r.command
	,CASE /* Uses statement start and end offset to figure out what statement is running */
		WHEN r.[statement_start_offset] > 0
			THEN
				/* The start of the active command is not at the beginning of the full command text */
				CASE r.[statement_end_offset]
					WHEN - 1
						THEN
							/* The end of the full command is also the end of the active statement */
							SUBSTRING(t.TEXT, (r.[statement_start_offset] / 2) + 1, 2147483647)
					ELSE
						/* The end of the active statement is not at the end of the full command */
						SUBSTRING(t.TEXT, (r.[statement_start_offset] / 2) + 1, (r.[statement_end_offset] - r.[statement_start_offset]) / 2)
					END
		ELSE
			/* 1st part of full command is running */
			CASE r.[statement_end_offset]
				WHEN - 1
					THEN
						/* The end of the full command is also the end of the active statement */
						RTRIM(LTRIM(t.[text]))
				ELSE
					/* The end of the active statement is not at the end of the full command */
					LEFT(t.TEXT, (r.[statement_end_offset] / 2) + 1)
				END
		END AS [executing_statement]
	,t.[text] AS [parent_batch]
	,s.[program_name]
	,r.query_hash
	,r.query_plan_hash
	,r.dist_statement_id
	,r.[label]
	,s.client_interface_name
	,r.[sql_handle]
	,c.client_net_address
	,c.connection_id
FROM	sys.dm_exec_requests r
CROSS APPLY sys.[dm_exec_sql_text](r.[sql_handle]) t
JOIN	sys.dm_exec_sessions s ON r.session_id = s.session_id
JOIN	sys.dm_exec_connections c ON s.session_id = c.session_id
JOIN	sys.databases d ON d.database_id = r.database_id
WHERE	r.dist_statement_id != '00000000-0000-0000-0000-000000000000'
AND	r.session_id <> @@SPID
AND	s.program_name NOT IN ('QueryInsights','DMS')
</pre></p>
""")


StatementMeta(, 56ed4fc1-00c2-4248-bba5-123781108efd, 3, Finished, Available, Finished)